In [34]:
from bs4 import BeautifulSoup
import requests
import json

POKEDEX_BASE_URL = "https://pokemondb.net/pokedex/"

national_pokedex = requests.get("https://pokemondb.net/pokedex/national")

# Using the "infocard" class to identify pokemon in the list
pokemon_count = len(BeautifulSoup(national_pokedex.text, "html.parser").find_all('div', attrs={"class": "infocard"}))


In [35]:
pokemon = {}

for dig in range(1, pokemon_count + 1):
    new_data = {}
    index = f"{dig:03d}"

    try:
        pokemon_page = requests.get(POKEDEX_BASE_URL + index)
    except requests.exceptions.RequestException:
        continue
    
    poke_soup = BeautifulSoup(pokemon_page.text, 'html.parser')

    type_obj = poke_soup.table.find_all('a', {'class': "type-icon"})
    new_data['types'] = [t.text for t in type_obj]

    height = poke_soup.find(text="Height").parent.parent.td.text.split('\xa0m')[0]
    new_data['height_meters'] = height

    weight = poke_soup.find(text="Weight").parent.parent.td.text.split('\xa0kg')[0]
    new_data['weight_kg'] = weight

    img_link = poke_soup.find('a', {'data-title': True})
    if img_link:
        new_data['img_link'] = img_link['href']

    pokemon[index] = new_data

with open("data/pokemon_data.json", "w") as f:
    json.dump(pokemon, f)
